In [158]:
import numpy as np
import math
import csv
from statistics import mode
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD

In [159]:
# SET50 (unrealize)
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEAUTY', 'BEM', 'BH', 'BJC', 'BPP', 'BTS', 'CBG',
        'CENTEL', 'CPALL', 'CPF', 'CPN', 'DTAC', 'EA', 'EGCO',
        'GLOBAL', 'GPSC', 'HMPRO', 'INTUCH', 'IRPC', 'IVL', 'KBANK',
        'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'MTLS', 'PSH', 'PTT',
        'PTTEP', 'PTTGC', 'ROBINS', 'SAWAD', 'SCB', 'SCC', 'SPRC',
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TPIPP', 'TRUE', 'TU', 'WHA']

In [168]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[2] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = (float(row[2]) + float(row[3]))/2.0
                if temp_value != 0:
                    unrealize = (temp - temp_value) / temp_value
                else:
                    unrealize = 0
                unrealize = ["{0:.3f}".format(unrealize)]
                stock_data.append(unrealize)
                temp_value = temp
                
    return stock_data

def findMean(x):
    sums = 0.0
    for i in x:
        sums += float(i[0])
        
    returns = []
    sums = "{0:.3f}".format(sums/len(x))
    returns.append(sums)
    return returns

def mean_square_error(actual, expected):
    length = len(actual)
    sums_error = 0.0
    for i, j in zip(actual, expected):
        sums_error += (i[0] - j[0])**2
    
    return sums_error / length

In [223]:
stock_datas = [ load_stock_data(x, 2007, 2017) for x in SET50 ]
# delete date **
e = mode([ len(x) for x in stock_datas])
rounds = int(e / 22)
delete_indexs = []
for index, element in enumerate(stock_datas):
    if len(element) < rounds * 22:
        delete_indexs.append(index)
        
for i in reversed(delete_indexs):
    del stock_datas[i]

In [250]:
model = Sequential()
    
model.add(Conv2D(64, (3, 3), input_shape=(len(stock_datas), 22, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3 ,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))   
      
model.add(Flatten())  
model.add(Dense(512))  
model.add(Activation('relu'))  
model.add(Dropout(0.5))
model.add(Dense(1))# equal symbols

sgd = SGD(lr = 0.01, momentum = 0.9, decay= 1e-6, nesterov = True)

model.compile(loss='mean_squared_error', optimizer=sgd)

In [251]:
train_index = int(rounds * 0.9)

In [252]:
target = []
index = 1
while index < rounds:
    temp = [stock_datas[i][index * 22] for i in range(len(stock_datas))]
    target.append(findMean(temp))
    index += 1
    
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_test = y[train_index: rounds].astype('float32')

print(y_train.shape, y_test.shape)

(108, 1) (12, 1)


In [253]:
data = []
count = 0
index = 0
while index < rounds - 1:
    temp = [i[count : count + 22] for i in stock_datas]
    data.append(temp)
    count += 22
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_test = x[train_index: rounds].astype('float32')

print(x_train.shape, x_test.shape)

(108, 29, 22, 1) (12, 29, 22, 1)


In [254]:
epochs = 100

model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = epochs, shuffle = True, batch_size = 100, verbose= 1)

Train on 108 samples, validate on 12 samples
Epoch 1/100
108/108 [==============================] - 1s 13ms/step - loss: 1.3799e-04 - val_loss: 5.9980e-06
Epoch 2/100
108/108 [==============================] - 0s 352us/step - loss: 1.3278e-04 - val_loss: 6.0441e-06
Epoch 3/100
108/108 [==============================] - 0s 343us/step - loss: 1.4684e-04 - val_loss: 7.3019e-06
Epoch 4/100
108/108 [==============================] - 0s 352us/step - loss: 1.8163e-04 - val_loss: 9.2517e-06
Epoch 5/100
108/108 [==============================] - 0s 361us/step - loss: 2.3035e-04 - val_loss: 6.1425e-06
Epoch 6/100
108/108 [==============================] - 0s 352us/step - loss: 1.3655e-04 - val_loss: 5.8430e-06
Epoch 7/100
108/108 [==============================] - 0s 352us/step - loss: 1.4933e-04 - val_loss: 6.2400e-06
Epoch 8/100
108/108 [==============================] - 0s 333us/step - loss: 1.8078e-04 - val_loss: 6.9132e-06
Epoch 9/100
108/108 [==============================] - 0s 343us/step

Epoch 74/100
108/108 [==============================] - 0s 334us/step - loss: 1.2709e-04 - val_loss: 1.0133e-05
Epoch 75/100
108/108 [==============================] - 0s 352us/step - loss: 1.0081e-04 - val_loss: 1.0177e-05
Epoch 76/100
108/108 [==============================] - 0s 334us/step - loss: 9.1962e-05 - val_loss: 9.8457e-06
Epoch 77/100
108/108 [==============================] - 0s 361us/step - loss: 1.2691e-04 - val_loss: 9.8619e-06
Epoch 78/100
108/108 [==============================] - 0s 333us/step - loss: 1.1174e-04 - val_loss: 8.9017e-06
Epoch 79/100
108/108 [==============================] - 0s 352us/step - loss: 9.7190e-05 - val_loss: 8.5353e-06
Epoch 80/100
108/108 [==============================] - 0s 343us/step - loss: 9.2476e-05 - val_loss: 8.1585e-06
Epoch 81/100
108/108 [==============================] - 0s 334us/step - loss: 1.0191e-04 - val_loss: 8.3746e-06
Epoch 82/100
108/108 [==============================] - 0s 343us/step - loss: 1.1421e-04 - val_loss: 8.2

In [255]:
predict = model.predict(x_test, verbose = 0)

print(predict)

[[  4.43944242e-03]
 [  1.56791846e-03]
 [  1.23050064e-04]
 [  1.06722268e-03]
 [  1.43525749e-03]
 [  1.35611917e-03]
 [  1.19774602e-03]
 [  8.66791059e-04]
 [  1.23048550e-03]
 [  6.91056601e-04]
 [ -6.86041312e-06]
 [  1.62228348e-03]]


In [256]:
mse = mean_square_error(classes, y_test)
print(mse)

6.41176494518e-06
